In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [88]:
#read file: USD_CAD_M15_2005_202206
FILE_NAME_LIST = [
    'USD_CHF_M5_2005_202206','USD_CHF_M15_2005_202206','USD_CHF_M30_2005_202206',
    'AUD_CAD_M5_2005_202206','AUD_CAD_M15_2005_202206','AUD_CAD_M30_2005_202206',
    'AUD_JPY_M5_2005_202206','AUD_JPY_M15_2005_202206','AUD_JPY_M30_2005_202206',
    'AUD_USD_M5_2005_202206','AUD_USD_M15_2005_202206','AUD_USD_M30_2005_202206',
    'EUR_CAD_M5_2005_202206','EUR_CAD_M15_2005_202206','EUR_CAD_M30_2005_202206',
    'EUR_JPY_M5_2005_202206','EUR_JPY_M15_2005_202206','EUR_JPY_M30_2005_202206',
    'NZD_USD_M5_2005_202206','NZD_USD_M15_2005_202206','NZD_USD_M30_2005_202206',
    'USD_JPY_M5_2005_202206','USD_JPY_M15_2005_202206','USD_JPY_M30_2005_202206'
]

#change directroy
data_ls = []
for i in FILE_NAME_LIST:
    file = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\{i}.csv', encoding='utf-8')
    data_ls.append(file)

In [4]:
len(data_ls)

24

In [5]:
#check for any missing value
file.isnull().sum()

datetime    0
open        0
high        0
low         0
close       0
dtype: int64

In [107]:
data = file.copy()
#use lastest 250 row of data for visual and pattern detect

data

,datetime,open,high,low,close
0,2005-01-02T18:30:00.000000000Z,102.540,102.540,102.490,102.510
1,2005-01-02T19:00:00.000000000Z,102.580,102.730,102.560,102.710
2,2005-01-02T19:30:00.000000000Z,102.710,102.710,102.640,102.690
3,2005-01-02T20:00:00.000000000Z,102.680,102.710,102.670,102.700
4,2005-01-02T20:30:00.000000000Z,102.690,102.770,102.660,102.730
...,...,...,...,...,...
221447,2022-06-30T21:00:00.000000000Z,135.720,135.756,135.709,135.742
221448,2022-06-30T21:30:00.000000000Z,135.742,135.762,135.706,135.745
221449,2022-06-30T22:00:00.000000000Z,135.742,135.799,135.726,135.732
221450,2022-06-30T22:30:00.000000000Z,135.730,135.767,135.718,135.738


In [7]:
fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

fig.show()


In [106]:
def check_ascending_order2(nums):
    if  nums == sorted(nums):
        return True
    else:
        return False

def check_descending_order2(nums):
    if  nums == sorted(nums,reverse=True):
        return True
    else:
        return False

def check_ascending_order(nums):
    count = 0
    if len(nums) != 1:
        for i in range(len(nums)):
            if nums[i] >= nums[0]:
                count+=1  
    if count == len(nums):
        return True
    else:
        return False

def check_descending_order(nums):
    count = 0
    if len(nums) != 1:
        for i in range(len(nums)):
            if nums[i] <= nums[0]:
                count+=1
    if count == len(nums):
        return True
    else:
        return False       
    
def list_handle(nested_list):
    # return the first and end point of the list
    result = []
    for ls in nested_list:
        if len(ls) > 0:
            new_ls = [ls[0], ls[len(ls)-1]]
            result.append(new_ls)
        else:
            continue
    return result

def converge(high, low):
    #trading volume decrease
    if (high[-1] - low[-1]) < high[0] - low[0]:
        return True
    else:
        return False

def rising_wedge(df,high,low, datetime,length = 5):
    #length: minimum number of candlestick bar that satisfy the conditions
    high_ls = []
    rw_datetime_ls = []
    low_ls = []
    try:
        df = df.reset_index()
    except:
        pass
    df['Rising Wedge'] = 0

    for i in range(len(high)):
    # rising wedge satisfy 3 conditions:
    # high and low both are rising
    # converge (trading volumne is decreasing)
        if check_ascending_order(high[i:i+length]) and check_ascending_order(low[i:i+length]) and converge(high[i:i+length],low[i:i+length]):
            high_ls.append(high[i:i+length])
            low_ls.append(low[i:i+length])
            rw_datetime_ls.append(datetime[i:i+length])

            # mark the dataframe falling wedge = 1, otherwise 0
            df['Rising Wedge'][i:i+length] = 1
            
    return df, high_ls, rw_datetime_ls, low_ls
        
def falling_wedge(df,high,low, datetime,length = 5):
    #length: minimum number of candlestick bar that satisfy the conditions
    low_ls = []
    fw_datetime_ls = []
    high_ls = []
    try:
        df = df.reset_index()
    except:
        pass
    df['Falling Wedge'] = 0

    for i in range(len(high)):
    # falling wedge satisfy 3 conditions:
    # high and low both are falling
    # converge (trading volumne is decreasing)
        if  check_descending_order(high[i:i+length]) and check_descending_order(low[i:i+length]) and converge(high[i:i+length],low[i:i+length]):
            low_ls.append(low[i:i+length])
            high_ls.append(high[i:i+length])
            fw_datetime_ls.append(datetime[i:i+length])

            # mark the dataframe falling wedge = 1, otherwise 0
            df['Falling Wedge'][i:i+length] = 1

    return df ,high_ls, fw_datetime_ls, low_ls

def analysis_rising_wedge(data):
# collect results into a list for visualization
    final_a, final_b, final_c = [],[],[]
    for i in range(len(data['Rising Wedge'])-1):
        if data['Rising Wedge'][i] == 1:
            final_a.append(data['low'][i])
            final_b.append(data['high'][i])
            final_c.append(data['datetime'][i])
            if data['Rising Wedge'][i+1] == 0:
                # print('r')
                final_a.append('X')
                final_b.append('X')
                final_c.append('X')

    temp =[]
    result_high, result_high_bottom, result_datetime = [],[],[]

    for i in final_a:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_high.append(temp)
            temp = []
    temp =[]
    for i in final_b:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_high_bottom.append(temp)
            temp = []
    temp =[]
    for i in final_c:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_datetime.append(temp)
            temp = []
            
    # connect the first and last number in the list for visualization
    result_high = list_handle(result_high)
    result_datetime = list_handle(result_datetime)
    result_high_bottom = list_handle(result_high_bottom)
    print(str(len(result_high)) + ' rising wedge patterns')

    return result_high, result_datetime, result_high_bottom

def analysis_falling_wedge(data):
# collect results into a list for visualization
    # data = data.reset_index()
    final_a, final_b, final_c = [],[],[]
    for i in range(len(data['Falling Wedge'])-1):
        if data['Falling Wedge'][i] == 1:
            final_a.append(data['low'][i])
            final_b.append(data['high'][i])
            final_c.append(data['datetime'][i])

            if data['Falling Wedge'][i+1] == 0:
                final_a.append('X')
                final_b.append('X')
                final_c.append('X')
                
    temp =[]
    result_low, result_low_bottom, result_datetime = [],[],[]
    for i in final_a:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_low.append(temp)
            temp = []
    temp =[]
    for i in final_b:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_low_bottom.append(temp)
            temp = []
    temp =[]
    for i in final_c:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_datetime.append(temp)
            temp = []

    # connect the first and last number in the list for visualization
    result_low = list_handle(result_low)
    result_datetime = list_handle(result_datetime)
    result_low_bottom = list_handle(result_low_bottom)
    print(str(len(result_low)) + ' falling wedge patterns')
    
    return result_low, result_datetime, result_low_bottom

In [94]:
idx = 0
num_ls = []
for data in data_ls:
    high = list(data['high'])
    low = list(data['low'])
    datetime =  list(data['datetime'])
    data, high_ls, datetime_ls, low_ls = rising_wedge(data,high,low,datetime,length = 5)
    print(f'{FILE_NAME_LIST[idx]}: ', end = '')
    result_high, result_datetime, result_low = analysis_rising_wedge(data)
    num_ls.append(len(result_datetime))
    idx += 1

rising_wedge_table = pd.DataFrame(pd.Series(num_ls, index=FILE_NAME_LIST))

USD_CHF_M5_2005_202206: 14528 rising wedge patterns
USD_CHF_M15_2005_202206: 4301 rising wedge patterns
USD_CHF_M30_2005_202206: 2065 rising wedge patterns
AUD_CAD_M5_2005_202206: 14614 rising wedge patterns
AUD_CAD_M15_2005_202206: 4487 rising wedge patterns
AUD_CAD_M30_2005_202206: 2190 rising wedge patterns
AUD_JPY_M5_2005_202206: 15867 rising wedge patterns
AUD_JPY_M15_2005_202206: 5086 rising wedge patterns
AUD_JPY_M30_2005_202206: 2474 rising wedge patterns
AUD_USD_M5_2005_202206: 15836 rising wedge patterns
AUD_USD_M15_2005_202206: 4865 rising wedge patterns
AUD_USD_M30_2005_202206: 2285 rising wedge patterns
EUR_CAD_M5_2005_202206: 14059 rising wedge patterns
EUR_CAD_M15_2005_202206: 4156 rising wedge patterns
EUR_CAD_M30_2005_202206: 2077 rising wedge patterns
EUR_JPY_M5_2005_202206: 14320 rising wedge patterns
EUR_JPY_M15_2005_202206: 4431 rising wedge patterns
EUR_JPY_M30_2005_202206: 2182 rising wedge patterns
NZD_USD_M5_2005_202206: 14518 rising wedge patterns
NZD_USD_M15_

In [101]:
rising_wedge_table

,0
USD_CHF_M5_2005_202206,14528
USD_CHF_M15_2005_202206,4301
USD_CHF_M30_2005_202206,2065
AUD_CAD_M5_2005_202206,14614
AUD_CAD_M15_2005_202206,4487
AUD_CAD_M30_2005_202206,2190
AUD_JPY_M5_2005_202206,15867
AUD_JPY_M15_2005_202206,5086
AUD_JPY_M30_2005_202206,2474
AUD_USD_M5_2005_202206,15836


In [97]:
idx = 0
num_ls = []

for data in data_ls:
    high = list(data['high'])
    low = list(data['low'])
    datetime =  list(data['datetime'])
    data, fw_high_ls, fw_datetime_ls, fw_low_ls = falling_wedge(data,high,low,datetime,length = 5)
    print(f'{FILE_NAME_LIST[idx]}: ' , end = '' )
    result_high, result_datetime, result_low = analysis_falling_wedge(data)
    num_ls.append(len(result_datetime))
    idx += 1

falling_wedge_table2 = pd.DataFrame(pd.Series(num_ls, index=FILE_NAME_LIST))

USD_CHF_M5_2005_202206: 14455 falling wedge patterns
USD_CHF_M15_2005_202206: 4135 falling wedge patterns
USD_CHF_M30_2005_202206: 1934 falling wedge patterns
AUD_CAD_M5_2005_202206: 13845 falling wedge patterns
AUD_CAD_M15_2005_202206: 4045 falling wedge patterns
AUD_CAD_M30_2005_202206: 1983 falling wedge patterns
AUD_JPY_M5_2005_202206: 14006 falling wedge patterns
AUD_JPY_M15_2005_202206: 3952 falling wedge patterns
AUD_JPY_M30_2005_202206: 1700 falling wedge patterns
AUD_USD_M5_2005_202206: 14936 falling wedge patterns
AUD_USD_M15_2005_202206: 4280 falling wedge patterns
AUD_USD_M30_2005_202206: 1989 falling wedge patterns
EUR_CAD_M5_2005_202206: 14195 falling wedge patterns
EUR_CAD_M15_2005_202206: 4258 falling wedge patterns
EUR_CAD_M30_2005_202206: 2164 falling wedge patterns
EUR_JPY_M5_2005_202206: 13160 falling wedge patterns
EUR_JPY_M15_2005_202206: 3776 falling wedge patterns
EUR_JPY_M30_2005_202206: 1689 falling wedge patterns
NZD_USD_M5_2005_202206: 13891 falling wedge pa

In [100]:
falling_wedge_table2

,0
USD_CHF_M5_2005_202206,14455
USD_CHF_M15_2005_202206,4135
USD_CHF_M30_2005_202206,1934
AUD_CAD_M5_2005_202206,13845
AUD_CAD_M15_2005_202206,4045
AUD_CAD_M30_2005_202206,1983
AUD_JPY_M5_2005_202206,14006
AUD_JPY_M15_2005_202206,3952
AUD_JPY_M30_2005_202206,1700
AUD_USD_M5_2005_202206,14936


In [110]:
################# Visualization Purpose ####################
data = data[0:200]
high = list(data['high'])
low = list(data['low'])
datetime =  list(data['datetime'])
data, high_ls, datetime_ls, low_ls = falling_wedge(data,high,low,datetime,length = 5)

result_high, result_datetime, result_low = analysis_falling_wedge(data)

fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

for i in range(len(result_high)):
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_high[i] , mode='lines', marker_color = 'red', showlegend=False, name = 'Falling Wedge'))
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_low[i], mode='lines', marker_color = 'red',showlegend=False))

fig.update_layout(title="Falling Wedge")
fig.show()

10 falling wedge patterns


In [111]:
################# Visualization Purpose ####################
high = list(data['high'])
low = list(data['low'])
datetime =  list(data['datetime'])
data, high_ls, datetime_ls, low_ls = rising_wedge(data,high,low,datetime,length = 5)

result_high, result_datetime, result_low = analysis_rising_wedge(data)

fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

for i in range(len(result_high)):
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_high[i] , mode='lines', marker_color = 'green', showlegend=False, name = 'Rising Wedge'))
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_low[i], mode='lines', marker_color = 'green',showlegend=False))

fig.update_layout(title="Rising Wedge")
fig.show()

10 rising wedge patterns
